In [1]:
#coding=utf-8
#the first line is necessary to run this code on server

##########################################
# Accounting Factors translated from SAS 
# December 03 2019
# Created by Xinyu LIU
##########################################

import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from pandas.core.frame import DataFrame
from scipy import stats
import datetime
from matplotlib.backends.backend_pdf import PdfPages

###################
# Connect to WRDS #
###################
conn = wrds.Connection(wrds_username='dachxiu')
#make it a constant portal by creating ppass

###################
# Compustat Block #
###################
comp = conn.raw_sql("""
                    select 
                    f.cusip as cnum, c.gvkey, datadate, datadate as
                    datadate_a, fyear, c.cik, sic as sic2, sic, naics, 
                    sale, revt, cogs, xsga, xrd, xad, ib, ebitda, ebit, nopi, spi, pi, txp, 
                    ni, txfed, txfo, txt, xint, 
                    capx, oancf, dvt, ob, gdwlia, gdwlip, gwo, 
                    rect, act, che, ppegt, invt, at, aco, intan, ao, ppent, gdwl, fatb, fatl, 
                    lct, dlc, dltt, lt, dm, dcvt, cshrc, dcpstk, pstk, ap, lco, lo, drc, drlt, txdi,
                    ceq, scstkc, emp, csho, /*addition*/
                    pstkrv, pstkl, txditc, datadate as year, /*market*/
                    abs(prcc_f) as prcc_f, csho*prcc_f as mve_f, /*HXZ*/
                    am, ajex, txdb, seq, dvc, dvp, dp, dvpsx_f, mib, ivao, ivst, sstk, prstkc, 
                    dv, dltis, dltr, dlcch, oibdp, dvpa, tstkp, oiadp, xpp, xacc, re, ppenb, 
                    ppenls, capxv, fopt, wcap
                    from comp.names as c, comp.funda as f
                    where 
                    f.gvkey=c.gvkey
                    /*get consolidated, standardized, industrial format statements*/
                    and f.indfmt='INDL' 
                    and f.datafmt='STD'
                    and f.popsrc='D'
                    and f.consol='C'
                    and datadate >= '01/01/2015'
                    """)

# Due to limited functionality caused by using sql on python, need to manually make up the modifiers work in SAS sql
comp.cnum=comp.cnum.replace(' ','').str.slice(0, 6)
comp.sic2=comp.sic2+'12'
comp.datadate=pd.to_datetime(comp.datadate)
comp.year = comp.datadate.dt.year
comp=comp.dropna(subset=['at','prcc_f','ni'])

# create preferrerd stock
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])
#manipulate ps data in the sequense of redemption, liquidating and total value, last resolution is 0

comp['txditc']=comp['txditc'].fillna(0)

# create book equity
comp['be']=comp['ceq']+comp['txditc']-comp['ps']
comp['be']=np.where(comp['be']>0,comp['be'],None)
#comp['be']=np.where(comp['be']>0, comp['be'], np.nan)
#Book value of equity equals to Stockholders Equity + Deferred Tax - Preferred Stocks 
#set nan value for book equity that is less than 0

# number of years in Compustat
comp=comp.sort_values(by=['gvkey','datadate']).drop_duplicates()

# number of years in Compustat
comp['count']=comp.groupby(['gvkey']).cumcount()
#Sort DataFrame by column gvkey and datadate
#Mark cumulative number of each gvkey as of that row, starting from 0


###################
# CRSP Block      #
###################
# sql similar to crspmerge macro
crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.ticker, b.ncusip, b.shrcd, b.exchcd, b.siccd,
                      a.prc, a.ret, a.retx, a.shrout, a.vol
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '07/01/2015' and '06/30/2018'
                      and b.exchcd between 1 and 3
                      and b.shrcd between 10 and 11
                      """) 
#b.dlprc does not exist

# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['date']=pd.to_datetime(crsp_m['date'])
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)
#The 1 in MonthEnd just specifies to move one step forward to the next date that's a month end.

# add delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """)
#MSEDELIST		CRSP Monthly Stock Event - Delisting
#DLRET 	Num	8	Delisting Return,DLRET is the return of the security after it is delisted. 
#It is calculated by comparing a value after delisting against the price on the security's last trading date. 
#The value after delisting can include a delisting price or the amount from a final distribution.
#DLSTDT 	Num	8	Delisting Date,DLSTDT contains the date (in YYMMDD format) of a security's last price on the current exchange.

#process dlret
dlret.permno=dlret.permno.astype(int)
dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

#merge dlret and crsp_m
crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])
#crsp and dlret share the same column names: permno and jdate

#process crsp
crsp['dlret']=crsp['dlret'].fillna(0)
crsp['ret']=crsp['ret'].fillna(0)
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

# calculate market equity
crsp['me']=crsp['prc'].abs()*crsp['shrout']
# Newly added columns in parallel with SAS code, not necessary to be here 
# lag absolute close price, market cap
crsp['prca']=crsp['prc'].abs()
crsp['lprc']=crsp.groupby(['permno','permco'])['prca'].shift(1)
crsp['lme']=crsp.groupby(['permno','permco'])['me'].shift(1)
#market equity equals to price of stock times shares of outstanding

#process crsp
crsp=crsp.drop(['dlret','dlstdt'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me']).drop_duplicates()

### Aggregate Market Cap ###
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()
# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()
# join by jdate/maxme to find the permno
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])
# drop me column and replace with the sum me
crsp1=crsp1.drop(['me'], axis=1)
# join with sum of me to get the correct market cap info
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])
# sort by permno and date and also drop duplicates
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()
# important to have a duplicate check


# keep December market cap
crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month
decme=crsp2[crsp2['month']==12]
decme=decme[['permno','date','jdate','me','year']].rename(columns={'me':'dec_me'})

### July to June dates
crsp2['ffdate']=crsp2['jdate']+MonthEnd(-6)
crsp2['ffyear']=crsp2['ffdate'].dt.year
crsp2['ffmonth']=crsp2['ffdate'].dt.month
crsp2['1+retx']=1+crsp2['retx']
crsp2=crsp2.sort_values(by=['permno','date'])

# cumret by stock
crsp2['cumretx']=crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()
#cumprod returns the product of the year in this case, which is the cumulative return as time goes by

# lag cumret
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(1)

# lag market cap
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(1)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp2['lme']=np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

# baseline me
mebase=crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})

# merge result back together
crsp3=pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])

decme['year']=decme['year']+1
decme=decme[['permno','year','dec_me']]

# Info as of June
crsp3_jun = crsp3[crsp3['month']==6]

crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno','year'])

# Because I haven't reach the end of the code so will temporarily leave this subslicing open
# crsp_jun=crsp_jun[['permno','date', 'jdate', 'shrcd','exchcd','retadj','me','wt','cumretx','mebase','lme','dec_me']]
crsp_jun=crsp_jun.sort_values(by=['permno','jdate']).drop_duplicates()
crsp_jun=crsp_jun.drop(columns=['count'])

#######################
# CCM Block           #
#######################
ccm=conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and linkprim in ('P', 'C')
                  """)
#CCMXPF_LINKTABLE		CRSP/COMPUSTAT Merged - Link History w/ Used Flag
#lpermno 	Num	8	Historical CRSP PERMNO Link to COMPUSTAT Record
# linktype 	Char	2	Link Type Code,
# Link Type Code is a 2-character code providing additional detail on the usage of the link data available.
# linkprim 	Char	1	Primary Link Marker
# linkdt 	Num	8	First Effective Date of Link
# linkenddt 	Num	8	Last Effective Date of Link

ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))
#attention: pd.to.datetime does not convert today(M8[ns]) into format '%Y\%m\%d', need to go with ccm[].dt.date
# if using the code below there will be warning on server
ccm['linkenddt']=ccm['linkenddt'].dt.date
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])

ccm1=pd.merge(comp,ccm,how='left',on=['gvkey'])
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

# set link date bounds
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]
# Subject to further adjustment in the future 
ccm2=ccm2.drop(columns=['datadate_a','linktype','linkdt','linkenddt'])
# ccm2=ccm2[['gvkey','permno','datadate','yearend','jdate','be','op','inv','count']]

# # link comp and crsp
# Note: Different from SAS code, I left merge CCM2 to CRSP_JUN
# It could be exactly the same if using the form below
# ccm_jun pd.merge( ccm2,crsp_jun, how='left', on=['permno', 'jdate'])
ccm_jun=pd.merge(crsp_jun, ccm2, how='inner', on=['permno', 'jdate'])

# ccm_data is the SAS parallel
ccm_data=pd.merge(ccm2,crsp_jun, how='left', on=['permno', 'jdate'])
#filtering out prc==nan and dec_me==0
ccm_jun=ccm_jun[ccm_jun.dec_me!=0]
ccm_jun['beme']=ccm_jun['be']*1000/ccm_jun['dec_me']

# drop duplicates
ccm_jun=ccm_jun.sort_values(by=['permno','date']).drop_duplicates()
ccm_jun=ccm_jun.sort_values(by=['gvkey','date']).drop_duplicates()

# Note: Different from SAS, Python count start from zero, will see if I need to add 1 to better serve the need
ccm_jun['count']=ccm_jun.groupby(['gvkey']).cumcount()

# Parallel to the cleaning step for 'dr'
ccm_jun['dr']=np.where(ccm_jun.drc.notna() & ccm_jun.drlt.notna(),ccm_jun.drc+ccm_jun.drlt,None)
ccm_jun['dr']=np.where(ccm_jun.drc.notna() & ccm_jun.drlt.isna(),ccm_jun.drc,ccm_jun['dr'])
ccm_jun['dr']=np.where(ccm_jun.drc.isna() & ccm_jun.drlt.notna(),ccm_jun.drlt,ccm_jun['dr'])
# Parallel to the cleaning step for 'dc'
ccm_jun['dc']=np.where(ccm_jun.dcvt.isna() & ccm_jun.dcpstk.notna() & ccm_jun.pstk.notna() & (ccm_jun.dcpstk>ccm_jun.pstk),\
                       ccm_jun.dcpstk-ccm_jun.pstk,None)
ccm_jun['dc']=np.where(ccm_jun.dcvt.isna() & ccm_jun.dcpstk.notna() & ccm_jun.pstk.isna(),\
                       ccm_jun.dcpstk,ccm_jun['dr'])
ccm_jun['dc']=np.where(ccm_jun.dc.isna(), ccm_jun.dcvt, ccm_jun['dr'])
ccm_jun['xint']=ccm_jun['xint'].fillna(0)
ccm_jun['xsga']=ccm_jun['xsga'].fillna(0)

ccm_jun=ccm_jun.sort_values(by=['permno','date']).drop_duplicates()

Loading library list...
Done


In [20]:
#######################
# more clean-up and create first pass of variables           #
#######################
#create simple-just annual Compustat variables

ccm_jun['ep']=ccm_jun.ib/ccm_jun.mve_f
ccm_jun['cashpr']=(ccm_jun.mve_f+ccm_jun.dltt-ccm_jun['at'])/ccm_jun.che
ccm_jun['dy']=ccm_jun.dvt/ccm_jun.mve_f
ccm_jun['lev']=ccm_jun['lt']/ccm_jun.mve_f
ccm_jun['sp']=ccm_jun.sale/ccm_jun.mve_f
ccm_jun['roic']=(ccm_jun.ebit-ccm_jun.nopi)/(ccm_jun.ceq+ccm_jun['lt']-ccm_jun.che)
ccm_jun['rd_sale']=ccm_jun.xrd/ccm_jun.sale
ccm_jun['sp']=ccm_jun.sale/ccm_jun.mve_f

#Deleting duplicated columns
#ccm_jun = ccm_jun.loc[:,~ccm_jun.columns.duplicated()]

# treatment for lagged terms
ccm_jun['lagat']=ccm_jun.groupby(['permno'])['at'].shift(1)
ccm_jun['lagcsho']=ccm_jun.groupby(['permno'])['csho'].shift(1)
ccm_jun['laglt']=ccm_jun.groupby(['permno'])['lt'].shift(1)
ccm_jun['lagact']=ccm_jun.groupby(['permno'])['act'].shift(1)
ccm_jun['lagche']=ccm_jun.groupby(['permno'])['che'].shift(1)
ccm_jun['lagdlc']=ccm_jun.groupby(['permno'])['dlc'].shift(1)
ccm_jun['lagtxp']=ccm_jun.groupby(['permno'])['txp'].shift(1)
ccm_jun['laglct']=ccm_jun.groupby(['permno'])['lct'].shift(1)
ccm_jun['laginvt']=ccm_jun.groupby(['permno'])['invt'].shift(1)
ccm_jun['lagemp']=ccm_jun.groupby(['permno'])['emp'].shift(1)
ccm_jun['lagsale']=ccm_jun.groupby(['permno'])['sale'].shift(1)
ccm_jun['lagib']=ccm_jun.groupby(['permno'])['ib'].shift(1)
ccm_jun['lag2at']=ccm_jun.groupby(['permno'])['at'].shift(2)
ccm_jun['lagrect']=ccm_jun.groupby(['permno'])['rect'].shift(1)
ccm_jun['lagcogs']=ccm_jun.groupby(['permno'])['cogs'].shift(1)
ccm_jun['lagxsga']=ccm_jun.groupby(['permno'])['xsga'].shift(1)
ccm_jun['lagppent']=ccm_jun.groupby(['permno'])['ppent'].shift(1)
ccm_jun['lagdp']=ccm_jun.groupby(['permno'])['dp'].shift(1)
ccm_jun['lagxad']=ccm_jun.groupby(['permno'])['xad'].shift(1)
ccm_jun['lagppegt']=ccm_jun.groupby(['permno'])['ppegt'].shift(1)
ccm_jun['lagceq']=ccm_jun.groupby(['permno'])['ceq'].shift(1)
ccm_jun['lagcapx']=ccm_jun.groupby(['permno'])['capx'].shift(1)
ccm_jun['lag2capx']=ccm_jun.groupby(['permno'])['capx'].shift(2)
ccm_jun['laggdwl']=ccm_jun.groupby(['permno'])['gdwl'].shift(1)
ccm_jun['lagdvt']=ccm_jun.groupby(['permno'])['dvt'].shift(1)
ccm_jun['lagob']=ccm_jun.groupby(['permno'])['ob'].shift(1)
ccm_jun['lagaco']=ccm_jun.groupby(['permno'])['aco'].shift(1)
ccm_jun['lagintan']=ccm_jun.groupby(['permno'])['intan'].shift(1)
ccm_jun['lagao']=ccm_jun.groupby(['permno'])['ao'].shift(1)
ccm_jun['lagap']=ccm_jun.groupby(['permno'])['ap'].shift(1)
ccm_jun['laglco']=ccm_jun.groupby(['permno'])['lco'].shift(1)
ccm_jun['laglo']=ccm_jun.groupby(['permno'])['lo'].shift(1)
ccm_jun['lagdr']=ccm_jun.groupby(['permno'])['dr'].shift(1)
ccm_jun['lagxrd']=ccm_jun.groupby(['permno'])['xrd'].shift(1)
ccm_jun['lagni']=ccm_jun.groupby(['permno'])['ni'].shift(1)
ccm_jun['lagdltt']=ccm_jun.groupby(['permno'])['dltt'].shift(1)



ccm_jun['agr']=np.where(ccm_jun['at'].isna() | ccm_jun.lagat.isna(), np.NaN, (ccm_jun.lagat-ccm_jun['at'])/ccm_jun.lagat)
ccm_jun['gma']=ccm_jun.revt-ccm_jun.cogs/ccm_jun.lagat
ccm_jun['chcsho']=ccm_jun.csho/ccm_jun.lagcsho -1
ccm_jun['lgr']=ccm_jun['lt']/ccm_jun.laglt -1
ccm_jun['acc']=(ccm_jun.ib-ccm_jun.oancf)/(ccm_jun['at']+ccm_jun.lagat) -1

ccm_jun['pctacc']=np.where(ccm_jun['ib']==0,(ccm_jun['ib']-ccm_jun['oancf'])/0.01, np.NaN)
ccm_jun['pctacc']=np.where(ccm_jun['oancf'].isna(),(ccm_jun['act']-ccm_jun['lagact']-(ccm_jun['che']-ccm_jun['lagche']))\
                           -(ccm_jun['lct']-ccm_jun['laglct']-(ccm_jun['dlc']-ccm_jun['lagdlc'])\
                            -(ccm_jun['txp']-ccm_jun['lagtxp'])-ccm_jun['dp'])/ccm_jun['ib'].abs(), ccm_jun['pctacc'])
ccm_jun['pctacc']=np.where(ccm_jun['oancf'].isna() & ccm_jun['ib']==0, (ccm_jun['act']-ccm_jun['lagact']-(ccm_jun['che']-ccm_jun['lagche']))\
                           -(ccm_jun['lct']-ccm_jun['laglct']-(ccm_jun['dlc']-ccm_jun['lagdlc'])\
                            -(ccm_jun['txp']-ccm_jun['lagtxp'])-ccm_jun['dp'])/0.01, ccm_jun['pctacc'])

ccm_jun['cfp']= (ccm_jun['ib']-(ccm_jun['act']-ccm_jun['lagact']-(ccm_jun['che']-ccm_jun['lagche'])))\
                -(ccm_jun['lct']-ccm_jun['laglct']-(ccm_jun['dlc']-ccm_jun['lagdlc'])\
                  -(ccm_jun['txp']-ccm_jun['lagtxp'])-ccm_jun['dp'])/ccm_jun['mve_f']
ccm_jun['cfp']=np.where(ccm_jun['oancf'].notna(),ccm_jun['oancf']/ccm_jun['mve_f'], ccm_jun['cfp'])
ccm_jun['absacc']=ccm_jun['acc'].abs()
ccm_jun['chinv']=2*(ccm_jun['invt']-ccm_jun['laginvt'])/(ccm_jun['at']+ccm_jun['lagat'])
ccm_jun['spii']=np.where((ccm_jun['spi']!=0)&ccm_jun['spi'].notna(), 1, 0)

ccm_jun['spi']=2*ccm_jun['spi']/(ccm_jun['at']+ccm_jun['lagat'])
ccm_jun['cf']=2*ccm_jun['oancf']/(ccm_jun['at']+ccm_jun['lagat'])

ccm_jun['cf']=np.where(ccm_jun['oancf'].isna(), (ccm_jun['ib']-(ccm_jun['act']-ccm_jun['lagact']-(ccm_jun['che']-ccm_jun['lagche'])))\
                -(ccm_jun['lct']-ccm_jun['laglct']-(ccm_jun['dlc']-ccm_jun['lagdlc'])\
                  -(ccm_jun['txp']-ccm_jun['lagtxp'])-ccm_jun['dp'])/((ccm_jun['at']+ccm_jun['lagat'])/2),ccm_jun['cf'])  
ccm_jun['hire']=ccm_jun['emp']-ccm_jun['lagemp']/ccm_jun['lagemp']
ccm_jun['hire']=np.where(ccm_jun['emp'].isna() | ccm_jun['lagemp'].isna(), 0, ccm_jun['hire'])

ccm_jun['sgr']=ccm_jun['sale']/ccm_jun['lagsale'] -1
ccm_jun['chpm']=ccm_jun['ib']/ccm_jun['sale']-ccm_jun['lagib']/ccm_jun['lagsale']
ccm_jun['chato']=(ccm_jun['sale']/((ccm_jun['at']+ccm_jun['lagat'])/2)) - (ccm_jun['lagsale']/((ccm_jun['lagat'])+ccm_jun['lag2at'])/2)
ccm_jun['pchsale_pchinvt']=((ccm_jun['sale']-(ccm_jun['lagsale']))/(ccm_jun['lagsale']))-((ccm_jun['invt']-(ccm_jun['laginvt']))/(ccm_jun['laginvt']))
ccm_jun['pchsale_pchrect']=((ccm_jun['sale']-(ccm_jun['lagsale']))/(ccm_jun['lagsale']))-((ccm_jun['rect']-(ccm_jun['lagrect']))/(ccm_jun['lagrect']))
ccm_jun['pchgm_pchsale']=(((ccm_jun['sale']-ccm_jun['cogs'])-((ccm_jun['lagsale'])-(ccm_jun['lagcogs'])))/((ccm_jun['lagsale'])-(ccm_jun['lagcogs'])))-((ccm_jun['sale']-(ccm_jun['lagsale']))/(ccm_jun['lagsale']))
ccm_jun['pchsale_pchxsga']=((ccm_jun['sale']-(ccm_jun['lagsale']))/(ccm_jun['lagsale']) )-((ccm_jun['xsga']\
        -(ccm_jun['lagxsga'])) /(ccm_jun['lagxsga']) )
ccm_jun['depr']=ccm_jun['dp']/ccm_jun['ppent']
ccm_jun['pchdepr']=((ccm_jun['dp']/ccm_jun['ppent'])-((ccm_jun['lagdp'])/(ccm_jun['lagppent'])))/((ccm_jun['lagdp'])/(ccm_jun['lagppent']))
ccm_jun['chadv']=np.log(1+ccm_jun['xad'])-np.log((1+(ccm_jun['lagxad'])))
ccm_jun['invest']=((ccm_jun['ppegt']-(ccm_jun['lagppegt'])) +  (ccm_jun['invt']-(ccm_jun['laginvt'])) ) / (ccm_jun['lagat'])
ccm_jun['invest']=np.where(ccm_jun['ppegt'].isna(), ((ccm_jun['ppent']-(ccm_jun['lagppent'])) +  (ccm_jun['invt']-(ccm_jun['laginvt'])) ) / (ccm_jun['lagat']), ccm_jun['invest'])
ccm_jun['egr']=((ccm_jun['ceq']-(ccm_jun['lagceq']))/(ccm_jun['lagceq']) )

# Note here instead of using count>=2, I use 1 to stay in line iwth python cumcount
# Also starting from here I'll keep the SAS in the notes for comparison and debug
    # 	if missing(capx) and count>=2 then
    # 		capx=ppent-lag(ppent);
    # 	pchcapx=(capx-lag(capx))/lag(capx);
    # 	grcapx=(capx-lag2(capx))/lag2(capx);
    # 	grGW=(gdwl-lag(gdwl))/lag(gdwl);
ccm_jun['capx']=np.where(ccm_jun['capx'].isna() & ccm_jun['count']>=1,ccm_jun['ppent']-(ccm_jun['lagppent']), ccm_jun['capx'])
ccm_jun['pchcapx']=ccm_jun['capx']-ccm_jun['lagcapx']/ccm_jun['lagcapx']
ccm_jun['grcapx']=ccm_jun['capx']-ccm_jun['lag2capx']/ccm_jun['lag2capx']
ccm_jun['grGW']=ccm_jun['gdwl']-ccm_jun['laggdwl']/ccm_jun['laggdwl']
    # 	if missing(gdwl) or gdwl=0 then
    # 		grGW=0;
    # 	if gdwl ne 0 and not missing(gdwl) and missing(grGW) then
    # 		grGW=1;
ccm_jun['grGW']=np.where(ccm_jun['gdwl'].isna() | ccm_jun['gdwl']==0, 0, ccm_jun['grGW'])  
ccm_jun['grGW']=np.where(ccm_jun['gdwl'].notna() & ccm_jun['gdwl']!=0 & ccm_jun['grGW'].isna(), 1, ccm_jun['grGW']) 
    # 	if (not missing(gdwlia) and gdwlia ne 0) or (not missing(gdwlip) and gdwlip ne 
    # 		0) or (not missing(gwo) and gwo ne 0) then
    # 			woGW=1;
ccm_jun['woGW']=np.where((ccm_jun['gdwlia'].notna()&ccm_jun['gdwlia']!=0)|(ccm_jun['gdwlip'].notna()&(ccm_jun['gdwlip']!=0))|\
                                                                          (ccm_jun['gwo'].notna()&ccm_jun['gwo']!=0) , 1, 0)
    #	tang=(che+rect*0.715+invt*0.547+ppent*0.535)/at;
ccm_jun['tang']=(ccm_jun['che']+ccm_jun['rect']*0.715+ccm_jun['invt']*0.547+ccm_jun['ppent']*0.535)/ccm_jun['at']

# 	if (2100<=sic<=2199) or (2080<=sic<=2085) or (naics in ('7132', '71312', 
# 		'713210', '71329', '713290', '72112', '721120')) then
# 			sin=1;
# 	else
# 		sin=0;

# 	if missing(act) then
# 		act=che+rect+invt;

# 	if missing(lct) then
# 		lct=ap;
ccm_jun['sic']=ccm_jun['sic'].astype(int)
ccm_jun['sin']=np.where(ccm_jun['sic'].between(2100,2199) | ccm_jun['sic'].between(2080,2085) | (ccm_jun['naics'].isin(['7132', '71312', \
                                                                             '713210', '71329', '713290', '72112', '721120'])), 1, 0)
ccm_jun['act']=np.where(ccm_jun['act'].isna(), ccm_jun['che']+ccm_jun['rect']+ccm_jun['invt'],ccm_jun['act'])
ccm_jun['lct']=np.where(ccm_jun['lct'].isna(), ccm_jun['ap'], ccm_jun['lct'])

# 	currat=act/lct;
# 	pchcurrat=((act/lct)-(lag(act)/lag(lct)))/(lag(act)/lag(lct));
# 	quick=(act-invt)/lct;
# 	pchquick=((act-invt)/lct - (lag(act)-lag(invt))/lag(lct) )/ 
# 		((lag(act)-lag(invt) )/lag(lct) );
# 	salecash=sale/che;
# 	salerec=sale/rect;
# 	saleinv=sale/invt;
# 	pchsaleinv=((sale/invt)-(lag(sale)/lag(invt)) ) / (lag(sale)/lag(invt));
# 	cashdebt=(ib+dp)/((lt+lag(lt))/2);
# 	realestate=(fatb+fatl)/ppegt;
    
ccm_jun['currat']=ccm_jun['act']/ccm_jun['lct']
ccm_jun['pchcurrat']=((ccm_jun['act']/ccm_jun['lct'])-((ccm_jun['lagact'])/(ccm_jun['laglct'])))/((ccm_jun['lagact'])/(ccm_jun['lct']))
ccm_jun['quick']=(ccm_jun['act']-ccm_jun['invt'])/ccm_jun['lct']
ccm_jun['pchquick']=((ccm_jun['act']-ccm_jun['invt'])/ccm_jun['lct'] - ((ccm_jun['lagact'])-(ccm_jun['laginvt']))/(ccm_jun['laglct']) )/ (((ccm_jun['lagact'])-(ccm_jun['laginvt']))/(ccm_jun['laglct']))
ccm_jun['salecash']=ccm_jun['sale']/ccm_jun['che']
ccm_jun['salerec']=ccm_jun['sale']/ccm_jun['rect']
ccm_jun['saleinv']=ccm_jun['sale']/ccm_jun['invt']
ccm_jun['pchsaleinv']=((ccm_jun['sale']/ccm_jun['invt'])-((ccm_jun['lagsale'])/(ccm_jun['laginvt'])) ) / ((ccm_jun['lagsale'])/(ccm_jun['laginvt']))
ccm_jun['cashdebt']=(ccm_jun['ib']+ccm_jun['dp'])/((ccm_jun['lt']+(ccm_jun['laglt']))/2)
ccm_jun['realestate']=(ccm_jun['fatb']+ccm_jun['fatl'])/ccm_jun['ppegt']

# 	if missing(ppegt) then
# 		realestate=(fatb+fatl)/ppent;
# 	if (not missing(dvt) and dvt>0) and (lag(dvt)=0 or missing(lag(dvt))) then
# 		divi=1;
# 	else
# 		divi=0;
# 	if (missing(dvt) or dvt=0) and (lag(dvt)>0 and not missing(lag(dvt))) then
# 		divo=1;
# 	else
# 		divo=0;
ccm_jun['realestate']=np.where(ccm_jun['ppegt'].isna(), (ccm_jun['fatb']+ccm_jun['fatl'])/ccm_jun['ppent'], ccm_jun['realestate'])
ccm_jun['divi']=np.where((ccm_jun['dvt'].notna() & ccm_jun['dvt']>0) & ((ccm_jun['lagdvt'])==0 | (ccm_jun['lagdvt'].isna())),1,0)
ccm_jun['divo']=np.where((ccm_jun['dvt'].isna() | ccm_jun['dvt']==0) & ((ccm_jun['lagdvt'])>0 & (ccm_jun['lagdvt'].notna())),1,0)

# 	obklg=ob/((at+lag(at))/2);
# 	chobklg=(ob-lag(ob))/((at+lag(at))/2);
ccm_jun['obklg']=ccm_jun['ob']/((ccm_jun['at']+(ccm_jun['lagat']))/2)
ccm_jun['chobklg']=(ccm_jun['ob']-(ccm_jun['lagob']))/((ccm_jun['at']+(ccm_jun['lagat']))/2)

# 	if not missing(dm) and dm ne 0 then
# 		securedind=1;
# 	else
# 		securedind=0;
# 	secured=dm/dltt;
# 	if not missing(dc) and dc ne 0 or (not missing(cshrc) and CSHRC ne 0) then
# 		convind=1;
# 	else
# 		convind=0;
# 	conv=dc/dltt;
ccm_jun['securedind']=np.where(ccm_jun['dm'].notna() &ccm_jun['dm']!=0, 1, 0)
ccm_jun['secured']=ccm_jun['dm']/ccm_jun['dltt']
ccm_jun['convind']=np.where((ccm_jun['dc'].notna() & ccm_jun['dc']!=0) | (ccm_jun['cshrc'].notna() & ccm_jun['cshrc']!=0) , 1, 0)
ccm_jun['dc']=ccm_jun['dc'].astype(float)
## There will be inf in the result
ccm_jun['conv']=ccm_jun['dc']/ccm_jun['dltt']

# 	grltnoa=((rect+invt+ppent+aco+intan+ao-ap-lco-lo)-(lag(rect)+lag(invt)+lag(ppent)+lag(aco)+lag(intan)+lag(ao)-lag(ap)-lag(lco)-lag(lo)) 
# 		-(rect-lag(rect)+invt-lag(invt)+aco-lag(aco)-(ap-lag(ap)+lco-lag(lco)) 
# 		-dp))/((at+lag(at))/2);
# 	chdrc=(dr-lag(dr))/((at+lag(at))/2);
ccm_jun['grltnoa']=((ccm_jun['rect']+ccm_jun['invt']+ccm_jun['ppent']+ccm_jun['aco']+ccm_jun['intan']+ccm_jun['ao']-ccm_jun['ap']-ccm_jun['lco']-ccm_jun['lo'])\
                    -((ccm_jun['lagrect'])+(ccm_jun['laginvt'])+(ccm_jun['lagppent'])+(ccm_jun['lagaco'])+(ccm_jun['lagintan'])+(ccm_jun['lagao'])-(ccm_jun['lagap'])\
                      -(ccm_jun['laglco'])-(ccm_jun['laglo'])) -(ccm_jun['rect']-(ccm_jun['lagrect'])+ccm_jun['invt']-(ccm_jun['laginvt'])+ccm_jun['aco']-(ccm_jun['lagaco'])\
                        -(ccm_jun['ap']-(ccm_jun['lagap'])+ccm_jun['lco']-(ccm_jun['laglco'])) -ccm_jun['dp']))/((ccm_jun['at']+(ccm_jun['lagat']))/2)
ccm_jun['chdrc']=(ccm_jun['dr']-(ccm_jun['lagdr']))/((ccm_jun['at']+(ccm_jun['lagat']))/2)

# 	if ((xrd/at)-(lag(xrd/lag(at))))/(lag(xrd/lag(at))) >.05 then
# 		rd=1;
# 	else
# 		rd=0;
# 	rdbias=(xrd/lag(xrd))-1-ib/lag(ceq);
# 	roe=ib/lag(ceq);
# additional process to carry on the next calculation
ccm_jun['xrd/lagat']=ccm_jun['xrd']/(ccm_jun['lagat'])
ccm_jun['lag(xrd/lagat)']=ccm_jun.groupby(['permno'])['xrd/lagat'].shift(1)
ccm_jun['rd']=np.where(((ccm_jun['xrd']/ccm_jun['at'])-ccm_jun['lag(xrd/lagat)'])/ccm_jun['lag(xrd/lagat)']>0.05, 1, 0)
ccm_jun['rdbias']=(ccm_jun['xrd']/(ccm_jun['lagxrd']))-1-ccm_jun['ib']/(ccm_jun['lagceq'])
ccm_jun['roe']=ccm_jun['ib']/(ccm_jun['lagceq'])

# 	ps_beme=coalesce(pstkrv, pstkl, pstk, 0);

# 	if missing(txditc) then
# 		txditc=0;
# 	BE=ceq + txditc - ps_beme;

# 	if BE<0 then
# 		BE=.;
# 	operprof=(revt-cogs-xsga-xint)/BE;

# 	if missing(revt) then
# 		operprof=.;

# 	if missing(cogs)=1 and missing(xsga)=1 and missing(xint)=1 then
# 		operprof=.;

# 	if missing(BE) then
# 		operprof=.;
ccm_jun['ps_beme']=np.where(ccm_jun['pstkrv'].isnull(), ccm_jun['pstkl'], ccm_jun['pstkrv'])
ccm_jun['ps_beme']=np.where(ccm_jun['ps_beme'].isnull(),ccm_jun['pstk'], ccm_jun['ps_beme'])
ccm_jun['ps_beme']=np.where(ccm_jun['ps_beme'].isnull(),0,ccm_jun['ps_beme'])
ccm_jun['txditc']=ccm_jun['txditc'].fillna(0)
ccm_jun['be']=ccm_jun['ceq']+ccm_jun['txditc']-ccm_jun['ps_beme']
ccm_jun['be']=np.where(ccm_jun['be']>0,ccm_jun['be'],np.NaN)
ccm_jun['operprof']=np.where(ccm_jun['be'].notna() & ccm_jun['revt'].notna() & (ccm_jun['cogs'].notna() | ccm_jun['xsga'].notna() | ccm_jun['xint'].notna()),\
                          (ccm_jun['revt']-ccm_jun['cogs']-ccm_jun['xsga']-ccm_jun['xint'])/ccm_jun['be'], np.nan)

#	ps=(ni>0)+(oancf>0)+(ni/at > lag(ni)/lag(at))+(oancf>ni)+(dltt/at < lag(dltt)/lag(at))+(act/lct > lag(act)/lag(lct)) 
#+((sale-cogs)/sale > (lag(sale)-lag(cogs))/lag(sale))+ (sale/at > lag(sale)/lag(at))+ (scstkc=0)
#!!! Not sure if the boolean adding is the same 
ccm_jun['ps']=(ccm_jun['ni']>0)+(ccm_jun['oancf']>0)+(ccm_jun['ni']/ccm_jun['at'] > (ccm_jun['lagni'])/(ccm_jun['lagat']))+(ccm_jun['oancf']>ccm_jun['ni'])+(ccm_jun['dltt']/ccm_jun['at'] < (ccm_jun['lagdltt'])/(ccm_jun['lagat']))\
                +(ccm_jun['act']/ccm_jun['lct'] > (ccm_jun['lagact'])/(ccm_jun['laglct'])) +((ccm_jun['sale']-ccm_jun['cogs'])/ccm_jun['sale'] > ((ccm_jun['lagsale'])-(ccm_jun['lagcogs']))/(ccm_jun['lagsale']))\
                + (ccm_jun['sale']/ccm_jun['at'] > (ccm_jun['lagsale'])/(ccm_jun['lagat']))+ (ccm_jun['scstkc']==0)

# 	if fyear<=1978 then
# 		tr=.48;

# 	if 1979<=fyear<=1986 then
# 		tr=.46;

# 	if fyear=1987 then
# 		tr=.4;

# 	if 1988<=fyear<=1992 then
# 		tr=.34;

# 	if 1993<=fyear then
# 		tr=.35;
# 	tb_1=((txfo+txfed)/tr)/ib;

# 	if missing(txfo) or missing(txfed) then
# 		tb_1=((txt-txdi)/tr)/ib;
# 	*they rank within industries;

def tr_fyear(row):
    if row['fyear']<=1978:
        value = 0.48
    elif row['fyear']<=1986:
        value = 0.46
    elif row['fyear']==1987:
        value = 0.4
    elif row['fyear']>=1988 and row['fyear']<=1992:
        value = 0.34
    elif row['fyear']>=1993:
        value = 0.35
    else:
        value=''
    return value
ccm_jun['tr']=ccm_jun.apply(tr_fyear, axis=1)
ccm_jun['tb_1']=((ccm_jun['txfo']+ccm_jun['txfed'])/ccm_jun['tr'])/ccm_jun['ib']
ccm_jun['tb_1']=np.where(ccm_jun['txfo'].isna() | ccm_jun['txfed'].isna(),((ccm_jun['txt']+ccm_jun['txdi'])/ccm_jun['tr'])/ccm_jun['ib'], ccm_jun['tb_1'])

# 	if (txfo+txfed>0 or txt>txdi) and ib<=0 then
# 		tb_1=1;
#!!! Caution that for condition, when using | and &, one must apply parenthesis 
ccm_jun['tb_1']=np.where(((ccm_jun['txfo']+ccm_jun['txfed'])>0 | (ccm_jun['txt']>ccm_jun['txdi'])) & (ccm_jun['ib']<=0), 1, ccm_jun['tb_1'])

# 	*variables that will be used in subsequent steps to get to final RPS;
# 	*--prep for for Mohanram (2005) score;
# 	roa=ni/((at+lag(at))/2);
# 	cfroa=oancf/((at+lag(at))/2);
ccm_jun['roa']=ccm_jun['ni']/((ccm_jun['at']+(ccm_jun['lagat']))/2)
ccm_jun['cfroa']=ccm_jun['oancf']/((ccm_jun['at']+(ccm_jun['lagat']))/2)

# 	if missing(oancf) then
# 		cfroa=(ib+dp)/((at+lag(at))/2);
# 	xrdint=xrd/((at+lag(at))/2);
# 	capxint=capx/((at+lag(at))/2);
# 	xadint=xad/((at+lag(at))/2);
ccm_jun['cfroa']=np.where(ccm_jun['oancf'].isna(),ccm_jun['ib']+ccm_jun['dp'] /((ccm_jun['at']+(ccm_jun['lagat']))/2), ccm_jun['cfroa'])
ccm_jun['xrdint']=ccm_jun['xrd']/((ccm_jun['at']+(ccm_jun['lagat']))/2)
ccm_jun['capxint']=ccm_jun['capx']/((ccm_jun['at']+(ccm_jun['lagat']))/2)
ccm_jun['xadint']=ccm_jun['xad']/((ccm_jun['at']+(ccm_jun['lagat']))/2)

# 	/*HXZ*/
# 	adm=xad/mve6b;
# 	gad=(xad-lag(xad))/lag(xad);
# 	rdm=xrd/mve6b;
# 	rds=xrd/sale;
# 	ol=(cogs+xsga)/at;
# 	rc_1=xrd+0.8*lag(xrd)+0.6*lag2(xrd)+0.4*lag3(xrd)+0.2*lag4(xrd);
    
#New lag terms for this section
ccm_jun['lag2xrd']=ccm_jun.groupby(['permno'])['lagxrd'].shift(1)
ccm_jun['lag3xrd']=ccm_jun.groupby(['permno'])['lag2xrd'].shift(1)
ccm_jun['lag4xrd']=ccm_jun.groupby(['permno'])['lag3xrd'].shift(1)

# Here I follow previous naming of mve6b as dec_me
ccm_jun['adm']=ccm_jun['xad']/ccm_jun['dec_me']  
ccm_jun['gad']=(ccm_jun['xad']-(ccm_jun['lagxad']))/(ccm_jun['lagxad'])
ccm_jun['rdm']=ccm_jun['xrd']/ccm_jun['dec_me']
ccm_jun['rds']=ccm_jun['xrd']/ccm_jun['sale']
ccm_jun['ol']=(ccm_jun['cogs']+ccm_jun['xsga'])/ccm_jun['at']
ccm_jun['rc_1']=ccm_jun['xrd']+0.8*(ccm_jun['lagxrd'])+0.6*(ccm_jun['lag2xrd'])+0.4*(ccm_jun['lag3xrd'])+0.2*(ccm_jun['lag4xrd'])

# 	cdd=dcvt/(dlc+dltt);
# 	roaq_a=ib/lag(at);
ccm_jun['cdd']=ccm_jun['dcvt']/(ccm_jun['dlc']+ccm_jun['dltt'])
ccm_jun['roaq_a']=ccm_jun['ib']/(ccm_jun['lagat'])
# roavol_1=std(roaq_a, lag(roaq_a), lag2(roaq_a), lag3(roaq_a), lag4(roaq_a)), lag5(roaq_a), lag6(roaq_a), lag7(roaq_a), lag8(roaq_a), lag9(roaq_a))
#!!! not pretty sure about this, need wider time span to test
#!!! I am using min_periods to temporarily bypass this issue (and it feels like the way SAS does this)
# must reindex in order to set back the value
ccm_jun['roavol_1']=ccm_jun.groupby(['permno'])['roaq_a'].rolling(10, min_periods=1).std().reset_index()['roaq_a']

# cs_1=(ib-(act-lag(act)-(lct-lag(lct))-(che-lag(che))+dlc-lag(dlc)))/lag(at)
ccm_jun['cs_1']=(ccm_jun['ib']-(ccm_jun['act']-(ccm_jun['lagact'])-(ccm_jun['lct']-(ccm_jun['laglct']))-(ccm_jun['che']-(ccm_jun['lagche']))+ccm_jun['dlc']-(ccm_jun['lagdlc'])))/(ccm_jun['lagat'])
# 	roavol_2=std(cs_1, lag(cs_1), lag2(cs_1), lag3(cs_1), lag4(cs_1), lag5(cs_1), 
# 		lag6(cs_1), lag7(cs_1), lag8(cs_1), lag9(cs_1));
# 	roavol_a=roavol_1/roavol_2;
ccm_jun['roavol_2']=ccm_jun.groupby(['permno'])['cs_1'].rolling(10, min_periods=1).std().reset_index()['cs_1']
ccm_jun['roavol_a']=ccm_jun['roavol_1']/ccm_jun['roavol_2']

# 	if missing(gdwl) then
# 		gdwl=0;

# 	if missing(intan) then
# 		intan=0;
# 	ala=che+0.75*(act-che)-0.5*(at-act-gdwl-intan);
# 	alm=ala/(at+prcc_f*csho-ceq);
# 	ob_a=ob/(.5*at+.5*lag(at));
# 	x_3=capx/sale;
# 	cinvest_a=x_3/((lag(x_3)+lag2(x_3)+lag3(x_3))/3)-1;
# 	dpia=(ppegt-lag(ppegt)+invt-lag(invt))/lag(at);

# 	if missing(dlc) then
# 		dlc=0;

# 	if missing(dltt) then
# 		dltt=0;

# 	if missing(mib) then
# 		mib=0;

# 	if missing(pstk) then
# 		pstk=0;

# 	if missing(ceq) then
# 		ceq=0;

ccm_jun['gdwl']=ccm_jun['gdwl'].fillna(0)
ccm_jun['intan']=ccm_jun['intan'].fillna(0)
ccm_jun['ala']=ccm_jun['che']+0.75*(ccm_jun['act']-ccm_jun['che'])-0.5*(ccm_jun['at']-ccm_jun['act']-ccm_jun['gdwl']-ccm_jun['intan'])
ccm_jun['alm']=ccm_jun['ala']/(ccm_jun['at']+ccm_jun['prcc_f']*ccm_jun['csho']-ccm_jun['ceq'])
ccm_jun['ob_a']=ccm_jun['ob']/(0.5*ccm_jun['at']+0.5*(ccm_jun['lagat']))
ccm_jun['x_3']=ccm_jun['capx']/ccm_jun['sale']
#New lag terms for this section
ccm_jun['lagx_3']=ccm_jun.groupby(['permno'])['x_3'].shift(1)
ccm_jun['lag2x_3']=ccm_jun.groupby(['permno'])['lagx_3'].shift(1)
ccm_jun['lag3x_3']=ccm_jun.groupby(['permno'])['lag2x_3'].shift(1)

ccm_jun['cinvest_a']=ccm_jun['x_3']/(((ccm_jun['lagx_3'])+(ccm_jun['lag2x_3'])+(ccm_jun['lag3x_3']))/3)-1

ccm_jun['dlc']=ccm_jun['dlc'].fillna(0)
ccm_jun['dltt']=ccm_jun['dltt'].fillna(0)
ccm_jun['mib']=ccm_jun['mib'].fillna(0)
ccm_jun['pstk']=ccm_jun['pstk'].fillna(0)
ccm_jun['ceq']=ccm_jun['ceq'].fillna(0)

In [27]:
# 	noa=((at-che)-(at-dlc-dltt-mib-pstk-ceq))/lag(at);
# 	dnoa=noa-lag(noa);
# 	pchcapx3=capx/lag3(capx)-1;
# 	x_4=dlc+dltt;
# 	cdi=log(x_4/lag5(x_4));
# 	ivg=invt/lag(invt)-1;
# 	dcoa=(act-lag(act)-(che-lag(che)))/lag(at);
# 	dcol=(lct-lag(lct)-(dlc-lag(dlc)))/lag(at);
# 	dwc=(dcoa-dcol)/lag(at);
# 	dnca=(at-act-ivao-(lag(at)-lag(act)-lag(ivao)))/lag(at);
# 	dncl=(lt-lct-dltt-(lag(lt)-lag(lct)-lag(dltt)))/lag(at);
# 	dnco=(dnca-dncl)/lag(at);
# 	dfin=(ivst+ivao-dltt-dlc-pstk-(lag(ivst)+lag(ivao)-lag(dltt)-lag(dlc)-lag(pstk)))/lag(at);
# 	ta=(dwc+dnco+dfin)/lag(at);
# 	dsti=(ivst-lag(ivst))/lag(at);
# 	dfnl=(dltt+dlc+pstk-(lag(dltt)+lag(dlc)+lag(pstk)))/lag(at);
# 	egr_hxz=(ceq-lag(ceq))/lag(at);

ccm_jun['noa']=((ccm_jun['at']-ccm_jun['che'])-(ccm_jun['at']-ccm_jun['dlc']-ccm_jun['dltt']-ccm_jun['mib']-ccm_jun['pstk']-ccm_jun['ceq']))/(ccm_jun['lagat'])
ccm_jun['lagnoa']=ccm_jun.groupby(['permno'])['noa'].shift(1)
ccm_jun['dnoa']=ccm_jun['noa']-(ccm_jun['lagnoa'])
ccm_jun['lag3capx']=ccm_jun.groupby(['permno'])['capx'].shift(3)
ccm_jun['pchcapx3']=ccm_jun['capx']/(ccm_jun['lag3capx'])-1
ccm_jun['x_4']=ccm_jun['dlc']+ccm_jun['dltt']
ccm_jun['lag5x_4']=ccm_jun.groupby(['permno'])['capx'].shift(5)
ccm_jun['cdi']=np.log(ccm_jun['x_4']/(ccm_jun['lag5x_4']))

ccm_jun['ivg']=ccm_jun['invt']/(ccm_jun['laginvt'])-1
ccm_jun['dcoa']=(ccm_jun['act']-(ccm_jun['lagact'])-(ccm_jun['che']-(ccm_jun['lagche'])))/(ccm_jun['lagat'])
ccm_jun['dcol']=(ccm_jun['lct']-(ccm_jun['laglct'])-(ccm_jun['dlc']-(ccm_jun['lagdlc'])))/(ccm_jun['lagat'])
ccm_jun['dwc']=(ccm_jun['dcoa']-ccm_jun['dcol'])/(ccm_jun['lagat'])
ccm_jun['lagivao']=ccm_jun.groupby(['permno'])['ivao'].shift(1)
ccm_jun['dnca']=(ccm_jun['at']-ccm_jun['act']-ccm_jun['ivao']-((ccm_jun['lagat'])-(ccm_jun['lagact'])-(ccm_jun['lagivao'])))/(ccm_jun['lagat'])
ccm_jun['dncl']=(ccm_jun['lt']-ccm_jun['lct']-ccm_jun['dltt']-((ccm_jun['laglt'])-(ccm_jun['laglct'])-(ccm_jun['lagdltt'])))/(ccm_jun['lagat'])
ccm_jun['dnco']=(ccm_jun['dnca']-ccm_jun['dncl'])/(ccm_jun['lagat'])
ccm_jun['lagivst']=ccm_jun.groupby(['permno'])['ivst'].shift(1)
ccm_jun['lagpstk']=ccm_jun.groupby(['permno'])['pstk'].shift(1)
ccm_jun['dfin']=(ccm_jun['ivst']+ccm_jun['ivao']-ccm_jun['dltt']-ccm_jun['dlc']-ccm_jun['pstk']-((ccm_jun['lagivst'])+(ccm_jun['lagivao'])-(ccm_jun['lagdltt'])-(ccm_jun['lagdlc'])-(ccm_jun['lagpstk'])))/(ccm_jun['lagat'])
ccm_jun['ta']=(ccm_jun['dwc']+ccm_jun['dnco']+ccm_jun['dfin'])/(ccm_jun['lagat'])
ccm_jun['dsti']=(ccm_jun['ivst']-(ccm_jun['lagivst']))/(ccm_jun['lagat'])
ccm_jun['dfnl']=(ccm_jun['dltt']+ccm_jun['dlc']+ccm_jun['pstk']-((ccm_jun['lagdltt'])+(ccm_jun['lagdlc'])+(ccm_jun['lagpstk'])))/(ccm_jun['lagat'])
ccm_jun['egr_hxz']=(ccm_jun['ceq']-(ccm_jun['lagceq']))/(ccm_jun['lagat'])


In [28]:
ccm_jun

,permno,permco,date,ticker,ncusip,shrcd,exchcd,siccd,prc,ret,...,dnca,dncl,dnco,lagivst,lagpstk,dfin,ta,dsti,dfnl,egr_hxz
0,10001,7953,2016-06-30,EGAS,36720410,11,2,4925.0,6.990000,-0.021008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,7953,2017-06-30,EGAS,36720410,11,2,4925.0,12.925000,0.023622,...,-0.023031,-5.169736e-03,-9.035095e-05,0.000,0.000,-0.018797,-9.541490e-05,0.000000,0.018797,-0.012808
2,10025,7975,2016-06-30,AEPI,00103110,11,3,3081.0,80.459999,0.340777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10026,7976,2016-06-30,JJSF,46603210,11,3,2052.0,119.269997,0.134218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10026,7976,2017-06-30,JJSF,46603210,11,3,2052.0,132.070007,0.018370,...,0.007177,5.764973e-03,1.900526e-06,0.000,0.000,0.036756,4.948270e-05,0.018224,0.000176,0.051223
5,10026,7976,2018-06-29,JJSF,46603210,11,3,2052.0,152.470001,0.079791,...,0.099992,2.020400e-02,1.009346e-04,13.539,0.000,0.021380,2.720340e-05,0.057653,-0.000450,0.056102
6,10028,7978,2016-06-30,DGSE,23323G10,11,2,5094.0,0.730000,0.377359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10028,7978,2017-06-30,DGSE,23323G10,11,2,5094.0,1.670000,0.012121,...,-0.168565,2.067192e-17,-1.043488e-02,0.000,0.000,0.241736,1.460174e-02,0.000000,-0.241736,0.126037
8,10028,7978,2018-06-29,DGSE,23323G10,11,2,5094.0,0.720000,-0.111111,...,0.001084,-2.586032e-17,8.396025e-05,0.000,0.000,0.049950,4.492360e-03,0.000000,-0.000929,0.143189
9,10032,7980,2016-06-30,PLXS,72913210,11,3,3670.0,43.200001,-0.016393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
